In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.genz.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
input_selkey = widgets.Dropdown(options=get_h5_keys(opts.info_fname, key='auto_qc'), disabled=False,
    description='Subject DWI to qc:',)
input_selkey.layout.width = '600px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'600px'), options=('/sub-genz906/ses-1/dwi/auto_qc', '/sub-genz510/ses-1/dwi/auto_qc', '/sub-genz508/ses-1/dwi/auto_qc', '/sub-genz506/ses-1/dwi/auto_qc', '/sub-genz503/ses-1/dwi/auto_qc', '/sub-genz502/ses-1/dwi/auto_qc', '/sub-genz501/ses-1/dwi/auto_qc', '/sub-genz308/ses-1/dwi/auto_qc', '/sub-genz305/ses-1/dwi/auto_qc', '/sub-genz303/ses-1/dwi/auto_qc', '/sub-genz302/ses-1/dwi/auto_qc', '/sub-genz301/ses-1/dwi/auto_qc', '/sub-genz205/ses-1/dwi/auto_qc', '/sub-genz203/ses-1/dwi/auto_qc', '/sub-genz201/ses-1/dwi/auto_qc', '/sub-genz104/ses-1/dwi/auto_qc', '/sub-genz103/ses-1/dwi/auto_qc'), value='/sub-genz906/ses-1/dwi/auto_qc')

In [8]:
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
all_dwi_auto_qc.dtypes

level_0               int64
x_bvec              float64
y_bvec              float64
z_bvec              float64
bvals                 int64
itopup                int64
alltopup_idx          int64
topup_qc              int64
itopdn                int64
topdn_qc              int64
auto_dwi_vol_idx      int64
auto_dwi_qc           int64
dwi_fname            object
topup_fname          object
topdn_fname          object
dtype: object

In [5]:
fslv_names

'/brainstudio/data/genz/sub-genz501/ses-1/dwi/sub-genz501_ses-1_dwi-topup_64dir-3sh-800-2000_1.nii -b 1,200 /brainstudio/data/genz/sub-genz501/ses-1/dwi/sub-genz501_ses-1_dwi-topup_6S0_1.nii -b 1,200 /brainstudio/data/genz/sub-genz501/ses-1/dwi/sub-genz501_ses-1_dwi-topdn_6S0_1.nii -b 1,200'

In [6]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
    with open('fslview_cmd.sh', 'w') as f:
        f.write('fslview ' + fslv_names + ' &\n')
#all_dwi_auto_qc['itopup_isnum'] = all_dwi_auto_qc['itopup'] == np.number
# limit to only active rows with mask

# all_dwi_auto_qc['itopup_isnum'] = all_dwi_auto_qc['itopup'].str.isnumeric()
# all_dwi_auto_qc['itopdn_isnum'] = all_dwi_auto_qc['itopdn'].str.isnumeric()
all_dwi_auto_qc.loc[all_dwi_auto_qc.itopup_isnum, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc.itopdn_isnum, 'itopdn_visqc'] = True
all_dwi_auto_qc.itopup_visqc.fillna(False, inplace=True)
all_dwi_auto_qc.itopdn_visqc.fillna(False, inplace=True)
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [5]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Box(children=(HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\xd9\x96\xaa\xba\x16@/]~\x8e\xd7S}_{\xc0\xff\x7f\xc2%-I\x08*\x16"\xd19\xc78g+"+ \x99\x95,B\xf8_\x0f\x00\x90\t\xff\xbbv\x01\x00\x00N\x05a\x01@6 ,\x00\xc8\x06\x84\x05\x00\xd9\x80\xb0\x00 \x1b\x10\x16\x00d\x03\xc2\x02\x80l@X\x00\x90\r\xf7#\xac\xae\xabV\xdaR\xd5u+m\xe9\xf2t\x96\xaa\xaaK\xbd\xa8\xedb\xea~v\xa7\x8a\xba\xaa\xe4w\x8bhy\xd96\xc3\xf2f\xb2<\xc1u\x0fW5\x1e\x81\xb8\xb4E\x17\x9d\x13\x89\x83\xb54\xd6\x915D]\x9d|\x1a\xaew\xc2\xde\x12\x08k9\x07\xcfK\xe1\x7fX\xca\xf3\xb3\x99T\xea\xf6\x12UX$7\x1a\x8aI\x98\xe8\xa7\n\xab\xf5\xaa{\xeb-/\x9bq\xf9Qe]NX\xc1.\x0b\xa9\xd0F\xefb\x18\xdd\xa3\xf1?V;\xe1\x8b)q\xb0\x16qtO\x85\xda\xf2\x89[K\x9e\xb0\x85\xfe;!\xc6%\xd3s\xec\x94%\xd9\x82\xb0\x96s\xf0\xbc,\xbc\x0f\x9bT5\x19N\x9f\xd3O\xda\x05\x14s\xc2\xaa\x15\xaa,\x95,H\xa1\x17\xd4\xee#\xd5\xf0H\xec\x94hL\xc3\xa4\xa9\xc2\xbd(\x82\xe5m\xfc\xbd\x88\xcb\t\xcb\xdf\xe5\xc2\x9a)\xaa\x96\x95\xdbKUZ\xef\x1c\x10N\xd6\x96\xc4\xc1Z\xc4\xd1=\x1d\x8ey\xd3\x9e\xea\x8d\xc4\t[:\xfd\xba\x83>=\xc7NY\x92/\x08k9\x87\xceK\xe1}(\xcf\x93\xba\xad\xbb(\xb2\\e\xfd*,\xd2\xa5\xf2b\xcb*\xddx\x1f\x85_\x98~]\x95\xb3\x15\xfau\xeb\xd5`)\xdcF\xb7LTS\xebH\xef\xe9b\xc2\xf2wY:\xb4i\xebjR\x1c?z\x11\xf8lxW\x87?\xcd\x81\x83u\x12G\xf7tXA\x9c\xbc\xb5\xe9\t[v\xfa\x8c\xaaF\xd1N\xcf\xb1S\x96d\x0c\xc2Z\xce\x81\xf3\xb2\xf0dT\x98*.\x1b*~+\xa4Y\xd0-8\x99b\xc6\x82\xfe^\xcb\n\xee\x15\xe4\x98\xb0\x9a\xa0\x91!\xc6\x1a\x1cl\xa6>Z\xb1/%,\x7f\x97\x85\x15U\x1b\xff\xccA\xf4\xc2oQ\r\xfb\'}\xeb\xf9\xed\xc0\xc1:\x89S\x84u\xfa\xd6\xa6\'lc\x1bI\x85\xdd\xdd\xe99v\xca\x92\x9cAX\xcb\x99;\xedj\xd3`\x1fW\xd3\x95A\x04.i\x97\xe41N#\n\xec\x13\xecu\x19\xbc;"\xac"\xea\x14\xc9\x1a\xac\x9a\'\xed\xa4\x1f\xe5\xaf6\xe5"\xc2\x8av\xb9v\xe2i\xa2\xe2\x04\xd1\x85gW!\x0fF`\xb0C\x07\xeb$.,,\xafD\xf67\x98\x9ec\xa7,\xc9\x99\x1b\x12\x96\xc9G\x9a\x13\xd7\xfb\xb9\xf5K\xf9\x7f!\xb3\xc8\x8d\xe9\xe6\x84\x886\xf8\xfa\xb8\xc1\xc2[C\xe6\x9e\x85w\xda\x85k\xd8\x04\x83Y=h\x93\xbb\xad\x0cg]\x9b<o\x8b8~\x14~\xba\x82#\x0c\x1c\x10\x9e\xf5A\x83\xe2\x88\xb0\xaa\xb8se\xebK\xd4\xaf\xa9\'i\xa3\xa8\xa4j\xcbzQ\xbcs\xad\xdb\x84\xfe\xa4p\xf6kMk`n\xa7\xa3]\x1ew\xb3\xac\xc3\x95g\x85\xa5#\x04G\xed\xc0\xc1\x1aK\xe4\xefs\xb0\x17^\xact\xa9;\xbf\xcc\xa9\xaf\x0em\xc6\xc2_=:a\xbd\x12\x98\x93kz\x8e\x9d\xb2$\xc0D\xe8g\xcf\xea\x9dq3\xc2\x1a\xf3\x91\xfaOfJXv\x9djZ\xe9\xc7Kg\xf6/\xae\xb0\x1b4k\x97\xee\xad\xfdi\'k\x14\x03\xe3I;\x9e^\xde\xcb\xe1;M\xea\x0fm\\|o\xe3"\xbd\x82\xff\xe5 p@X\x07\x0b\xbfSpXX\xe5\xb4\xab\xa7kp\x19\x95@\x94\xa58\xb8+r\xcb\xcd\xdc\xce\xd9\x8e\x8d.\x96\xcc\xb3\x14~\xac\xd9\x9d\x0ew\xb9\x9c\xef\xec\x04\xfb\xd5z5V{7\xd0\xed\x81\x83\x15T|\xa3\xb9\xf8\xf7\xb7\xb1\xe6J\xed\t+\xf9\xd5\xb6\xeb"a\x85\'l\xed\x19T\x87\x9a\x9ec\xa7,

In [10]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2J\xb7\x06\xd0G\x17\x93c\x18\xff\xc2\xf9\x0f\xe1I\x13H\xa7)U&Y\'\xef\xdek\xdd[\xa9\x02\x07\x10>!\x0c\xf0\xffn\x00A\xfc\xdfO\xcf\x00\xc0\xbb\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08C`\x01a\x08, \x0c\x81\x05\x84\xf1\xbb\x03\xab\xeb\xd2\x87\xa6\x94\xba\xeeCS\xfa^]\x96R\xd5\x8cO\xb5\xddVu{\xba@E\x95R?n\xb1y\xbei\xeb\xfb\xf3\xf5\xee\xf9\x03\xd7\xae\xaa\xf4X\xe2\xed\xdc\x15\xdd\xe6\xfd?X9gk}1DY\xa5u\xc9\x1f\xf5\xb9\xcd\xff\xdf!\xb0\xde\xf3r[-W/\xae\x1e\x15\xe3~^~h6\x9e\x16\xcd\xd6\xc14\x94=\x11X\xedb\xf7o\x17\xcf7\xf5\xe3\xf9/#\xeb{\x02\xab\xe9\xa3\xa0\xde\x17O\xab%\xab\x97+z\x98\xe9e0\x1d\xac\x9cS\xbe\\\xb2r\x98\xf2\x9bSk\xab\xea\xf4\x1c|\xa1\xa9\xea~k\xcb\xcb|\xb8\xf9?[\x91A\x08\xac\xf7\xbc\xdcV\x8b\xd5\x8b\x8bG\xcd\xbc;\xb5\x07\xa3\xfd\x9d\xe2Y`U\x83aoM\xfdNY\x8cOT\xf3K\xc3\x81\xc8\xc1\x02\x95\xf5t\xa0R\xa7\xf5\xce_\xac\x9e\xffjQ\xbe%\xb0rb\xd6\xdb\x98I\xf3R\rs\x97\x1e\xaf\x94s8g\x07+\xe7\x94/\x97\xec\xbe\x8e\xeb\xf6\xdd,\xf8\xf8z*\xe7\x8f\x95i-\x1dm\xfeOWd\x10\x02\xeb=\xaf\xb6\xaer\xf5\xe2\xe2Q3\xec1m\x956;\xce\'\x94\xc7s\xb4X\xe2\xe2>D\xbdxi=\xc2~\xf4~\x8a];n\xc7e\xbb\xd8\xa3\xfb\xc5\xa8\xc7O\xed\xe1P\xeb\x8b\xb3\xa9\xef\x08\xacz\\\x91\xdbs\xbcM\xb5>X\x8b\xe5\xa3j=\xfc\x8b\x95\xf3\x96/\x97\xec>\xc0\xfb9\xf0\xe9\xf54\xbc\x7f\xf7\x95\xd4\xaf\xabq\xd1\x0e6\xff\xe7+2\x08\x81\xf5\x9e\x17[W\xbf\xedw\x87\x8f\xea\xfcAV|\xbd\x9b\x9f\xb4.\xfa\xb0\\\xe2~\x0b^\x1c\x0e}\x15X\xf5\xea\xa0\xa3|\xec\xd1\xab\xc9T_\xee\xe8\xdf\x10X\xc54k\xfd1\xc4\xe6\x00oU\xadX~0\xdc\x87\xed\xf3u\xb1\xde_\xac\x9c\xb7\xbc\x13X\x1f\x9c\xdaIU\xde\xda\xe6E\xdbo\xfe/Vd\x10\x02\xeb=\xcf\xb6\xaej:\xe9;z\xd4<\xca\xb7\x1f=\xc4Z\x97YY-q\xb3z\xf4E`\x15\xdd\xfa$\xa9\xdf\xec\x87\xc3\x95vw^\xf5\xfa(\xe2\x1b\x02+\xe5\xdc)wK\xbd>\xba]\xa4i\xd9/\xfc*\xc1^\xad\x9cwg\xe3/\x07\xf8\xe3\x81\xdf\x9b\xde\xe3\x90xX\r\x87\xc7\xa3\xcfVd\x10\xc1\x03kj\xd3\x9e>E\x17o\xd0\xf8g\xff\xff\xb2oI\xae\xdb\xa3\x9d\xaclW\xa3?&X,\x86\xe8\xdb\x9f\xcb\xc5\xd6\xb5\x1e"7R\x1d=j\x1f\x9fb\xe5n\xdb)\xb6\xa57\x95\x0f\x86\x98\xad\xcb\xac\xac\xeb\xac\x0e0\xbe\x08\xac\xb4=\x08\xcc{\xf4\xe6<\xa7Z\x9ev\x1d\xcd\xe90\xe5\xf1\xa9\xed\xd2\xb5y\n\xe3\x0b\xc5\x1c~yU\x1d/\xf3b\xed\xd5\xdb\xeaO\x03k\x9c\xe4j-\xbdX9\xabwk^\xc6\xe5\\/k\x1d\xcff\xb7x[\x8e\xc6\xbc\x1f\x15\x17\x87\x03o\x07\x1f\xe6\xb3\xdf\xf2\xe6\xcdv\xb7e\xbf\x1c\xa6\\\x04\xd6j\xf3\x7f\xb5"\x87\xd7\xc7\x81oO7\xf7\x9f\x17:\xb0\x1em\xda\xe3\xc7\xe8Q`\xe5a\xd2~\xc7\x7f|}\x96?\x85\xcb<\xc1i\xe8f~\x98\xdf\xc1\xdd\x10\xc5\xdd\xfc\xf6\xae\x1fU\x8b=b\x93\x0f\xdb9_L\xb7|2\xc4r\xecU\x99\x95\xf5>Y,\x0f\xfc_\x07V\xb3?\xd5\x1b\xf7\xe8f3\x07e\xd3\xac\xc3\x7f;\xa7\xfd\x94\xebgK\xd7,\xe7\xaaoJ)\x96\xa5\x9e,\xf3b1\x8a\xed\xa9\xccj9\xda\xc5^8

In [13]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [14]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.0,0.0,0.0
1,2000,0,1,True,1.0,0.0,0.0
2,2000,1,0,True,0.0,1.0,0.0
3,2000,2,0,True,0.0,0.0,1.0
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,0,True,-0.025,-0.716,-0.698


In [11]:
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))